## `Non-Repetitive Parts Calculator` in Action!

**Authors** Ayaan Hossain and Howard Salis

**Updated** July 22, 2020

This `jupyter` notebook will demonstrate the usage of `NRP Calculator` and its [API](https://github.com/ayaanhossain/nrpcalc/blob/master/docs/DOCS.md) in designing some commonly used genetic parts, albeit in a hypothetical setting -- real design may include more realistic considerations and objectives. The purpose here is to deliberately demonstrate the different features of `NRP Calculator`.

If you [installed](https://github.com/ayaanhossain/nrpcalc#Installation) `nrpcalc` successfully, you have everything you need to follow along.

Click on the notebook title above "Non-Repetitive .. Action!" and press `Shift`+`Enter` together on each cell to follow along and see all the code in action.

First up, let's import `nrpcalc`

In [1]:
import nrpcalc

Hopefully, the import worked without throwing up any errors! If you face issues importing, please [open an issue](https://github.com/ayaanhossain/nrpcalc/issues). If everything worked fine, you're ready to follow along. If you do not understand a specific part of this notebook, either open an issue, or please reach the authors via Twitter or Email.

In [2]:
print(nrpcalc.__doc__)


Non-Repetitive Parts Calculator

Automated design and discovery of non-repetitive genetic
parts for engineering stable systems.

Version: 1.2.6

Authors: Ayaan Hossain <auh57@psu.edu>
         Howard Salis  <salis@psu.edu>

NRP Calculator offers two modes of operation:

- Finder Mode: Discover toolboxes of non-repetitive parts
               from a list of candidate parts

-  Maker Mode: Design toolboxes of non-repetitive parts
               based on sequence, structure and model
               constraints

Additionally, a 'background' object is available which can
be used to store background sequences against which parts
discovered or designed are ensured to be non-repetitive.

You can learn more about the two modes and background via
  print(nrpcalc.background.__doc__)
  print(nrpcalc.finder.__doc__)
  print(nrpcalc.maker.__doc__)



### Constraint Based Design of Genetic Parts

Genetic parts exhibit their activity through biophysical interactions that rely on `DNA` or `RNA` sequence motifs, the presence or absence of specific `RNA` structures, and/or higher-order sequence or structural characteristics. For example, a constitutive σ<sup>70</sup> _E. coli_ promoter sequence will have a high transcription initiation rate when it contains a conserved `-35` and `-10` hexamer, separated by a `17` base pair spacer. Likewise, a bacterial transcriptional terminator will have a high efficiency when it contains a fast-folding, stable `RNA` hairpin, followed by a `U`-rich tract.

Such essential characteristics can be flexibly distilled into a set of criteria that every generated genetic part sequence must satisfy. `NRP Calculator` `Maker Mode` accepts three types of genetic part constraints: a degenerate `DNA` or `RNA` sequence using the `IUPAC` code; an essential `RNA` secondary structure using `dot-parenthesis-x` notation; and a model-based constraint that can be customized to quantify the presence of higher-order interactions or to facilitate the synthesis and construction of the genetic system by excluding sequences (e.g. restriction sites and polymeric sequences). All three constraints may be simultaneously used to specify a set of genetic part sequences with desired functionalities.

As examples, **Supplementary Table 2** in our [publication](https://static-content.springer.com/esm/art%3A10.1038%2Fs41587-020-0584-2/MediaObjects/41587_2020_584_MOESM1_ESM.pdf) (see page 30) lists the design constraints and algorithm outcomes for a wide variety of genetic parts commonly used in Synthetic Biology, including minimal Pol II promoters, insulated ribosome binding sites, prokaryotic transcriptional terminators, and toehold `RNA` switches.

In one sense, these genetic part constraints are explicit hypotheses that distill one’s knowledge of gene regulation and biophysics into the simplest possible computable form. In another sense, they are a type of classifier that separates the genetic part sequence space into only two categories: sequences expected to have some amount of genetic part activity versus sequences expected to have minimal to none activity.

> **Note** The genetic part constraints are not a quantitative prediction of functional activity; experimental characterization is still needed to validate them. In general, it is advantageous to incorporate as much degeneracy into the constraints as possible to design larger toolboxes.

### Non-Repetitive σ<sup>70</sup> Promoters with CRISPRi with `Lmax`=`12`

We will first design `1000` brand new promoters for constitutive transcription in prokaryotes, divided into two toolboxes. We want the first toolbox to have `500` strong promoters, while for the second toolbox we want to design `500` promoters with variable strength. Additionally, we want these promoters to be CRISPRi repressible for our system logic (see [Reis et.al. (2019)](https://www.nature.com/articles/s41587-019-0286-9)). Importantly, we will use the findings from [Larson et.al. (2013)](https://www.nature.com/articles/nprot.2013.132) to design our CRISPRi.

#### Designing the First Toolbox

The sequence constraint for the first toolbox is defined for strong constitutive transcription, and a **PAM** is embedded in the `17`-bp spacer to repress transcription initiation via CRISPRi. To further enhance initiation, we will embed a `G+C`-rich motif upstream of `-35` hexamer.

In [3]:
tb1_seq_constraint = ('S'*5    + # G+C-rich motif in Upstream
                      'N'*15   + # remaining Upstream
                      'TTGACA' + # consensus -35 hexamer
                      'N'*6    + # first 6 bases of 17-bp Spacer
                      'CCN'    + # PAM (next 3 bases of 17-bp Spacer)
                      'N'*8    + # remaining 8 bases of 17-bp Spacer
                      'TATAAT' + # consensus -10 hexamer
                      'N'*6)     # discriminator

Let's review our sequence constraint.

In [4]:
tb1_seq_constraint

'SSSSSNNNNNNNNNNNNNNNTTGACANNNNNNCCNNNNNNNNNTATAATNNNNNN'

For promoters, we don't really have any structure constraint, so it can be all dots (i.e. we don't care about the secondary structure).

In [5]:
tb1_struct_constraint = '.'*len(tb1_seq_constraint)

In [6]:
tb1_struct_constraint

'.......................................................'

Next, we will define some functions to help us generate promoters that are compatible for our cloning purposes, i.e., we will want to prevent cutsites we use in our cloning from occuring in these promoters. While we can define functions to explicitly prevent our used cutsites (say, BamHI or XbaI), it would be better to prevent the occurence of any palindromic hexamer in our parts, which is usually a property of many restriction sites.

> **Note** `NRP Calculator` can optimize two types of functions - a local model function which is applied on a genetic part concurrently with addition of each nucleotide, and a global model function which is applied on a genetic part when it is fully constructed. The local model function must accept a single argument called `seq` (the partial sequence under construction) and return either `(True, None)` if the evaluation was satisfied, or `(False, index)` where `index` is a traceback location where nucleotide choices need to be altered. The global model function also must accept a single input `seq` (a fully constructed sequence), and return either `True` if an objective was met, or `False` otherwise.

We will now evaluate our new objective function to prevent palindromic hexamers in our designed promoters concurrently as each new base is added to a promoter towards completion when under construction (a local model function), starting with the addition of **sixth** base (base at `index`=`5` or equivalently, when `len(seq)`=`6`) which is when the promoter under construction qualifies for this specific evaluation (we are evaluating hexamers after all).

In [7]:
comptab = str.maketrans('ATGC', 'TACG') # our string translation table for complementng strings

In [8]:
def revcomp(seq):
    # reverse string, then return its complement
    return seq[::-1].translate(comptab)

In [9]:
assert revcomp('AAATTTGGGCCC') == 'GGGCCCAAATTT' # a quick test for revcomp function

In [10]:
assert revcomp('GGGAAATTTCCC') == 'GGGAAATTTCCC' # another quick check on palindromes

In [11]:
def prevent_cutsite(seq):
    # is our partial sequence long enough for evaluation?
    if len(seq) >= 6:
        # extract current 6-mer for evaluation
        hexamer = seq[-6:]
        if hexamer == revcomp(hexamer): # a palindrome!
            state = False      # objective failed
            index = len(seq)-6 # we need to go back 6 bases to alter current hexamer
            return (state, index)
        else:
            return (True, None) # sequence is free of palindromic hexamers
    # otherwise, pass for shorter sequences
    else:
        return (True, None)

The optimization done by this function is straight-forward. We check if a partial sequence under construction ends with a palindromic hexamer. If it does, then we ask `Maker` to go back `6` bases from our current index which is `len(seq)`-`6`, and start making alternate nucleotide choices starting **at** that location. If this function returns `True` for all locations starting at the sixth base, then naturally the complete part would be devoid of palindromic hexamers.

`Maker` takes care of calling this function with the addition of each base, so all we need to do in this function is evaluate only the "current" case, i.e., evaluate the hexamer ending at the current index.

> **Note** The traceback index should point to the location _starting at which_ nucleotide choices need to altered. For example, in the illustrated `prevent_cutsite` function above, we need to go back `6` bases from the current location if it is palindromic, and trace a new path through constraint space. This means, our traceback index should be `len(seq)` - `6` = `0` (the very beginning of our sequence) if our current sequence is of length `6`, and forms a palindromic hexamer.

In [12]:
assert prevent_cutsite('GGATCC') == (False, 0) # BamHI will be prevented

In [13]:
assert prevent_cutsite('TCTAGA') == (False, 0) # XbaI will be prevented

In [14]:
assert prevent_cutsite('GGGGGG') == (True, None)  # non-palindromic hexamer will pass our filter

At this point, we are ready to launch `NRP Calculator` `Maker Mode` to design some promoters for us!

In [15]:
toolbox1 = nrpcalc.maker(
    seed=1,                              # reproducible results
    seq_constr=tb1_seq_constraint,       # as defined above
    struct_constr=tb1_struct_constraint, # as defined above
    Lmax=12,                             # as stated in our goal
    target_size=500,                     # as stated in our goal
    part_type='DNA',                     # as stated in our goal
    local_model_fn=prevent_cutsite)      # as defined above


[Non-Repetitive Parts Calculator - Maker Mode]

[Checking Constraints]
  Sequence Constraint: SSSSSNNNNNNNNNNNNNNNTTGACANNNNNNCCNNNNNNNNNTATAATNNNNNN
 Structure Constraint: .......................................................
    Target Size      : 500 parts
           Lmax      : 12 bp
  Internal Repeats   : False

 Check Status: PASS

[Checking Arguments]
   Part Type : DNA
 Struct Type : mfe
  Synth Opt  : False
   Jump Count: 10
   Fail Count: 1000
 Output File : None

 Check Status: PASS

Constructing Toolbox:

 [part] 1, [13-mers] 43, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 2, [13-mers] 86, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 3, [13-mers] 129, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 4, [13-mers] 172, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 5, [13-mers] 215, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 6, [13-mers] 258, [iter time] 0.00s, [avg time] 0.00s, [to

 [part] 449, [13-mers] 19307, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 450, [13-mers] 19350, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 451, [13-mers] 19393, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 452, [13-mers] 19436, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 453, [13-mers] 19479, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 454, [13-mers] 19522, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 455, [13-mers] 19565, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 456, [13-mers] 19608, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 457, [13-mers] 19651, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 458, [13-mers] 19694, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 459, [13-mers] 19737, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 460, [13-mers] 19780, [iter time] 0.00s, [avg 

The console output shows that all our constraints passed initial checks, and were valid. `Maker` was able to design `500` promoters for us based on all given constraints without failure. Let's look at some of the sequences produced, and compare it with our sequence constraint.

In [16]:
toolbox1[0] # show the first part designed

'CCGCGAAATGAGAAGACTGCTTGACAAGTAATCCTACGAGTCGTATAATAGATAA'

In [17]:
toolbox1[499] # show the last part designed

'CGGGGGTCCGGGAGACCATTTTGACACCCAGCCCTTGGAATGATATAATACACAC'

In [18]:
tb1_seq_constraint

'SSSSSNNNNNNNNNNNNNNNTTGACANNNNNNCCNNNNNNNNNTATAATNNNNNN'

The construction looks good, but it's always a good idea to verify explicitly if our design objectives were met. We will define a verification function to check if our completely constructed promoters are indeed devoid of palindromic hexamers.

In [19]:
def final_cutsite_check(seq):
    # grab all hexamers from the sequence
    hexamers = [seq[i:i+6] for i in range(len(seq)-6+1)]
    # if any of the hexamers are panlindromic, our
    # design objective was clearly not met!
    for hexamer in hexamers:
        if hexamer == revcomp(hexamer):
            return False # we will reject this part
    # None of the hexamers were palindromic!
    return True  # we will accept this part

In [20]:
# We will loop through the toolbox, and ensure
# all our parts pass the new global check
for promoter in toolbox1.values():
    assert final_cutsite_check(promoter) == True

Every promoter seems to pass our check, so our design objective for the first toolbox was met. As we will see in the next section, the evaluation function `final_cutsite_check` could have been specified to `Maker` directly via `global_model_fn` parameter, which would automatically execute the evaluation on a part, after it was completely designed, and accept/reject it accordingly.

The benefit of passing this check as a global model function to `Maker` is that the algorithm can adjust the number of trials it needs depending on an auto-estimated probability of evaluation failure.

#### Designing the Second Toolbox - First Attempt

For our second toolbox of promoters, we want to design variable strength promoters. Our sequence constraint will change accordingly.

In [21]:
tb2_seq_constraint = 'N'*20 + 'TTGNNN' + 'N'*6 + 'CCN' + 'N'*6 + 'WW' + 'WWWWWW' + 'N'*6
#                     -----    ------     -----   ---     ----------     ------     ----
#                     UPS      -35        SPACER  PAM     SPACER         -10        DIS

Notice, we introduced degeneracy in the `-35` and opted for just weak bases (`A`/`T`)  in place of the `-10` hexamer. Additionally, the `-10` is also preceded by weak bases to potentially design promoters with various spacer region lengths ranging from `15` to `17` bp. We still retain the PAM in spacer for CRISPRi.

In [22]:
tb2_seq_constraint

'NNNNNNNNNNNNNNNNNNNNTTGNNNNNNNNNCCNNNNNNNWWWWWWWWNNNNNN'

Because things are more degenerate in our present constraint, we might be interested in preventing cryptic hexamers within our promoters.

This is easily done with another local model function, that identifies if a hexamer elsewhere within our promoter under construction, has fewer mismatches when compared to the consensus motifs than the ones placed (by `Maker`) at the intended `-35` and `-10` locations.

In [23]:
def hamming(x, y): # score mismatches between two strings
    return sum(x[i] != y[i] for i in range(min(len(x), len(y))))

In [24]:
assert hamming(x='000000', y='111111') == 6 # test case 1

In [25]:
assert hamming(x='000111', y='111111') == 3 # test case 2

In [26]:
def cryptic_hexamer(cx, hx, dt): # returns True if hx is a cryptic hexamer
    '''
    cx = consensus motif for either -35 or -10
    hx = current hexamer under evaluation
    dt = number of mismatches between cx and current
         motif placed at -35 or -10
    '''
    # if current hexamer (hx) is closer to consensus
    # motif (cm) than the actual selected motif used
    # at the intended location (dt), then we have a
    # cryptic promoter under construction (True)
    if hamming(cx, hx) < dt:
        return True
    return False

In [27]:
assert cryptic_hexamer(cx='TTGACA', hx='AAAAAA', dt=3) == False # test case 1

In [28]:
assert cryptic_hexamer(cx='TTGACA', hx='TTGAGA', dt=3) == True  # test case 2

In [29]:
def prevent_cryptic_promoter(seq, c35start, c10start, eval_index=None):
    '''
    seq - partial sequence under construction to be evaluated
    c35start - starting index of -35 hexamer
    c10start - starting index of -10 hexamer
    eval_index - the location ending at which a hexamer is to
                 be evaluated (default=None implies use the
                 last hexamer in current seq, i.e. ending at
                 len(seq))
    '''
    c35 = 'TTGACA' # defined -35 consensus
    c10 = 'TATAAT' # defined -10 consensus
    # sequence long enough to evaluate
    if len(seq) >= 6:
        # current index?
        end = len(seq)
        
        # which hexamer to evaluate?
        if eval_index is None: # no eval_index provided
            eval_index = end   # use the hexamer ending at current index (end)
        # otherwise use appropriate eval_index provided
        
        # extract current / appropriate hexamer
        hx = seq[eval_index-6:eval_index]
        
        # current hexamer at -35 or -10 location?
        # then skip evaluation for extracted hexamer
        if end == c35start+6:
            return (True, None) # skip -35 location
        if end == c10start+6:
            return (True, None) # skip -10 location
        
        # extract current -35 hexamer
        # if there is one
        s35 = None
        if end > c35start+6: # a -35 motif has been placed
            s35 = seq[c35start:c35start+6]
        # set -35 hexamer cutoff
        if s35 is None: # no -35 hexamer present yet
            d35 = 3 # default distance to prevent
        else:
            d35 = hamming(s35, c35) # actual distance to prevent
        # evaluate hx for -35 hexamer
        if cryptic_hexamer(cx=c35, hx=hx, dt=d35):
            return (False, end-6) # our current hexamer is a cryptic -35;
                                  # go back 6 bases
        
        # extract current -10 hexamer
        # if there is one
        s10 = None
        if end > c10start+6:
            s10 = seq[c10start:c10start+6]
        # set -10 hexamer cutoff
        if s10 is None: # no -10 hexamer present yet
            d10 = 3 # default distance to prevent
        else:
            d10 = hamming(s10, c10) # actual distance to prevent
        # evaluate hx for -35 hexamer
        if cryptic_hexamer(cx=c10, hx=hx, dt=d10):
            return (False, end-6) # our current hexamer is a cryptic -10;
                                  # go back 6 bases
        
        # both checks passed
        return (True, None) # part is OK
    # not long enough to evaluate
    return (True, None) # part is OK .. so far

In [30]:
# test case 1 - a partial sequence with last 6 bases very similar to the -35 consensus
assert prevent_cryptic_promoter(seq='GGGGGGGGTTGACT', c35start=20, c10start=20+6+17) == (False, 8)

In [31]:
# test case 2 - a partial sequence with last 6 bases very similar to the -10 consensus
assert prevent_cryptic_promoter(seq='GGGGGGGTATAGT', c35start=20, c10start=20+6+17) == (False, 7)

In [32]:
# test case 3 - a partial sequence with last 6 bases dissimilar to the both motifs
assert prevent_cryptic_promoter(seq='GGGGGGGGAAGATC', c35start=20, c10start=20+6+17) == (True, None)

The above local model function `prevent_cryptic_promoter` utilizes many smaller functions in order to make it's evaluation, which is fine. The only thing we need to take care of is `c35start` and `c10start` parameters, which are required for the function to work (note `eval_index` has a default value of `None` so we need not worry about it right now), although `Maker Mode` only works with local model functions that just takes in a single input - the partial sequence under construction. One obvious solution would be to hardcode all parameters apart from `seq` inside the local model function, but we don't really need to do that. Instead, we have two options.

The first option is to define a lambda function to wrap the above function like so.

In [33]:
prevent_cryptic = lambda seq: prevent_cryptic_promoter(seq=seq, c35start=20, c10start=43)

We could then set `local_model_fn=prevent_cryptic`, and our optimization would come through. This wrapping leaves the underlying `prevent_cryptic_promoter` free for more general use later. For example, it could used to power an additional global model function that checks the completely constructed part for cryptic hexamers by re-evaluating the hexamers at each index starting at `5` via the `eval_index` parameter (more on this a little later).

The second option is to define a wrapper function explicitly.

In [34]:
def prevent_cryptic(seq):
    return prevent_cryptic_promoter(seq=seq, c35start=20, c10start=43)

We now actually have two local model functions here: (1) the `prevent_cryptic` function and (2) the previously used `prevent_cutsite` function used for the first toolbox. In such multi-objective design scenerios, we would have to write a meta local model function, which would run these individual local model functions along with any specific parameters. Here's an example:

In [35]:
def variable_promoter_local(seq):
    # check the first objective
    outcome,index = prevent_cutsite(seq)
    # return traceback index if objective fails
    if outcome == False:
        return False,index
    # check the second objective
    outcome,index = prevent_cryptic_promoter(
        seq=seq,
        c35start=20,
        c10start=43)
    # return traceback index if objective fails
    if outcome == False:
        return False,index
    # every objective met .. good to go!
    return (True, None)

> **Note** It is important to be careful about string indexing when the functions become moderately complex. For example, Python uses `0`-indexing, which means the `-35` hexamer starts after the first `20` bases in the UPS region at index `20` (i.e. the `21`st base belongs to `-35`). It is also important to note when a model function should be evaluated. For example, if the evaluation logic requires at least an `8`-bp sequence, parts shorter than that length should be evaluated `True` to let the sequence grow long enough for evaluation.

> **Note** When there are multiple local objectives at play that evaluate properties of the partial sequence at an upstream location, it is often advantageous to return the traceback index that occurs earlier in the sequence position. For example, one thing we could do in a local model function is evaluate two objectives, and return `(False, min(index1, index2))` if both objective functions failed with different traceback locations, or just `(False, index1)` if only the first one failed and so on. We could also weigh the various objectives differently, and choose to return the most important traceback index first, rather than the second most important traceback index etc.

> **Note** It is possible to embed external models as evaluators into `NRP Calculator`. For example, rather than only preventing cryptic motifs as shown above, we could have additionally used a `Lasso` model, as described in [our publication](https://www.nature.com/articles/s41587-020-0584-2), to design promoters within specific dynamic ranges. We would load the model (unpickle) into memory, and for every promoter completely constructed by `Maker`, we would evaluate the predicted initation rate and only accept parts that satisfied our criteria (a global model function). We could further identify, using the model, which of the components (hexamers, spacer GC content etc.) prevented a part from being accepted, and returned a traceback index accordingly (i.e. convert the global check into a local model function) to explore nucleotide choices concurrently with part design.

> **Note** It is always a good idea to test the individual functions called by a meta local or global model function, using simple cases. Notice, how we have `assert`-ed a few test cases for the individual ones above.

Now that our meta local model function is ready, we can define a meta global model function that calls `final_cutsite_check` as well as `prevent_cryptic_promoter` like so.

In [36]:
def variable_promoter_global(seq):
    # check for cutsites post construction
    if not final_cutsite_check(seq):
        return False # cutsites found!
    
    # note: the following block could be
    # its own function, and called by this
    # meta global function
    
    # check for cyptic hexamers
    # starting at the 6th base
    for eval_index in range(6, len(seq)):
        # use the generalized evaluation function
        state, index = prevent_cryptic_promoter(
            seq=seq,
            c35start=20,
            c10start=43,
            eval_index=eval_index)
        # there is a cryptic hexamer ending
        # at the current location
        if state is False:
            return False
    
    # all checks passed!
    return True

With all our evaluators completed, we're ready to design our second toolbox of promoters. Let's call upon `Maker` to do our bidding.

In [37]:
toolbox2_attempt1 = nrpcalc.maker(
    seed=2,                                    # reproducible results
    seq_constr=tb2_seq_constraint,             # as defined above
    struct_constr=tb1_struct_constraint,       # same as toolbox1
    Lmax=12,                                   # as stated in our goal
    target_size=500,                           # as stated in our goal
    part_type='DNA',                           # as stated in our goal
    local_model_fn=variable_promoter_local,    # as defined above
    global_model_fn=variable_promoter_global)  # as defined above


[Non-Repetitive Parts Calculator - Maker Mode]

[Checking Constraints]
  Sequence Constraint: NNNNNNNNNNNNNNNNNNNNTTGNNNNNNNNNCCNNNNNNNWWWWWWWWNNNNNN
 Structure Constraint: .......................................................
    Target Size      : 500 parts
           Lmax      : 12 bp
  Internal Repeats   : False

 Check Status: PASS

[Checking Arguments]
   Part Type : DNA
 Struct Type : mfe
  Synth Opt  : False
   Jump Count: 10
   Fail Count: 1000
 Output File : None

 Check Status: PASS

Constructing Toolbox:

 [part] 1, [13-mers] 43, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 2, [13-mers] 86, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 3, [13-mers] 129, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 4, [13-mers] 172, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 5, [13-mers] 215, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 6, [13-mers] 258, [iter time] 0.00s, [avg time] 0.00s, [to

 [part] 140, [13-mers] 6020, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 141, [13-mers] 6063, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 142, [13-mers] 6106, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 143, [13-mers] 6149, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 144, [13-mers] 6192, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 145, [13-mers] 6235, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 146, [13-mers] 6278, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 147, [13-mers] 6321, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 148, [13-mers] 6364, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 149, [13-mers] 6407, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 150, [13-mers] 6450, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 151, [13-mers] 6493, [iter time] 0.00s, [avg time] 0.00s,

 [part] 271, [13-mers] 11653, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 272, [13-mers] 11696, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 273, [13-mers] 11739, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 274, [13-mers] 11782, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 275, [13-mers] 11825, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 276, [13-mers] 11868, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 277, [13-mers] 11911, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 278, [13-mers] 11954, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 279, [13-mers] 11997, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 280, [13-mers] 12040, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 281, [13-mers] 12083, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 282, [13-mers] 12126, [iter time] 0.00s, [avg 

 [part] 438, [13-mers] 18834, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 439, [13-mers] 18877, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 440, [13-mers] 18920, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 441, [13-mers] 18963, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 442, [13-mers] 19006, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 443, [13-mers] 19049, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 444, [13-mers] 19092, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 445, [13-mers] 19135, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 446, [13-mers] 19178, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 447, [13-mers] 19221, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 448, [13-mers] 19264, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 449, [13-mers] 19307, [iter time] 0.00s, [avg 

Let's review the new toolbox!

In [38]:
toolbox2_attempt1[0]

'GGGCTCCGAGCACTGGGGCTTTGCCTTGGCTTCCAGTTGGCTAAATAATGCGGAT'

In [39]:
toolbox2_attempt1[499]

'GTCCCGGTCTGGTCTTCGTATTGGCCGAGTGCCCGCAGAAGATTTTAATCTGAGT'

#### Designing the Second Toolbox - Second Attempt

The reason we called the previous section a **"First Attempt"** is because, the second toolbox we designed above is non-repetitive to itself, but not against `toolbox1` promoters designed apriori. To check non-repetitiveness in construction, we can use `Finder`.

In [40]:
# combine both toolboxes
promoters = list(toolbox1.values())
promoters.extend(toolbox2_attempt1.values())

In [41]:
# compute the number of non-repetitive promoters
non_repetitive_promoters = len(nrpcalc.finder(
    seq_list=promoters,
    Lmax=12))


[Non-Repetitive Parts Calculator - Finder Mode]

[Checking Constraints]
 Sequence List   : 1000 parts
          Lmax   : 12 bp
 Internal Repeats: False

 Check Status: PASS

[Checking Arguments]
   Vertex Cover: nrp2
   Output  File: None

 Check Status: PASS

Extracted 1000 unique sequences out of 1000 sequences in 0.001164 seconds

Written 1000 unique sequences out to ./86891db9-96e4-4190-a042-ccfa4544c35c/seq_list.txt in 0.001717 seconds

 [Sequence processing remaining] = 1    
 [Cliques inserted] = 989 

Built homology graph in 0.8326 seconds. [Edges = 13] [Nodes = 1000]
 [Intital Nodes = 1000] - [Repetitive Nodes = 0] = [Final Nodes = 1000]

 [+] Initial independent set = 0, computing vertex cover on remaining 0 nodes.
 [+] Vertex Cover Function: NRP 2-approximation
 [+] Dumping graph into: ./86891db9-96e4-4190-a042-ccfa4544c35c/repeat_graph.txt in 0.0026345252990722656 seconds

----------------------
Now running iteration: 0
----------------------

 Pendant checking is in progr

  [x] Isolated node 781 eliminated
  [x] Isolated node 408 eliminated
  [x] Isolated node 718 eliminated
  [x] Isolated node 345 eliminated
  [x] Isolated node 26 eliminated
  [x] Isolated node 987 eliminated
  [x] Isolated node 668 eliminated
  [x] Isolated node 311 eliminated
  [x] Isolated node 605 eliminated
  [x] Isolated node 232 eliminated
  [x] Isolated node 937 eliminated
  [x] Isolated node 874 eliminated
  [x] Isolated node 261 eliminated
  [x] Isolated node 555 eliminated
  [x] Isolated node 198 eliminated
  [x] Isolated node 903 eliminated
  [x] Isolated node 824 eliminated
  [x] Isolated node 467 eliminated
  [x] Isolated node 761 eliminated
  [x] Isolated node 148 eliminated
  [x] Isolated node 85 eliminated
  [x] Isolated node 790 eliminated
  [x] Isolated node 417 eliminated
  [x] Isolated node 727 eliminated
  [x] Isolated node 354 eliminated
  [x] Isolated node 35 eliminated
  [x] Isolated node 996 eliminated
  [x] Isolated node 677 eliminated
  [x] Isolated node 304

 [+] Current independent set size:  989
 [+] Potential nodes for expansion: 0 (projected independent set size: 989)
 [X] Cannot expand independent set, terminating.

Non-Repetitive Toolbox Size: 989


In [42]:
assert non_repetitive_promoters < 1000 # we're short of our goal ... some promoters were repetitive

As we can see, the final non-repetitive toolbox when both toolboxes are combined together has less than `1000` parts in it, which is short of our intended goal. This is where concept of **"background"** comes into play (check [DOCS](https://github.com/ayaanhossain/nrpcalc/blob/master/docs/DOCS.md) for `background` API details).

To create the second toolbox while ensuring it is non-repetitive to the first one, we will populate a temporary `background` object.

In [43]:
bkg = nrpcalc.background(
    path='./tmp_bkg', # we store the background on disk in the 'tmp_bkg' directory on current path
    Lmax=12)          # same as our intended design

In [44]:
bkg # checking background path and content, we see it has zero elements

kmerSetDB stored at ./tmp_bkg/ with 0 13-mers

In [45]:
# # we could add the promoters one-by-one
# for promoter in toolbox1.values():
#     bkg.add(promoter)

# or add it all in one-shot
bkg.multiadd(toolbox1.values())


[Background Processing]


  Adding Seq 268: GCCCGATTGA...

  Adding Seq 499: CGGGGGTCCG...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [46]:
bkg # now, background is populated with toolbox1 k-mers

kmerSetDB stored at ./tmp_bkg/ with 21500 13-mers

With `background` populated, we are ready to design our actual second toolbox such that it is indeed non-repetitive to the first toolbox, and therefore allowing both toolboxes to be used simultaneously. This is what we refer to as **toolbox chaining**. For example, you can chain a `Maker` job against a genome inserted in `background`. You can also use `background` with `Finder` jobs to succesively enlarge a central collection of parts from multiple toolboxes.

In [47]:
toolbox2 = nrpcalc.maker(
    seed=3,                                    # reproducible results
    seq_constr=tb2_seq_constraint,             # as defined above
    struct_constr=tb1_struct_constraint,       # same as toolbox1
    Lmax=12,                                   # as stated in our goal
    target_size=500,                           # as stated in our goal
    part_type='DNA',                           # as stated in our goal
    local_model_fn=variable_promoter_local,    # as defined above
    global_model_fn=variable_promoter_global,  # as defined above
    background=bkg)                            # as defined above


[Non-Repetitive Parts Calculator - Maker Mode]

[Checking Constraints]
  Sequence Constraint: NNNNNNNNNNNNNNNNNNNNTTGNNNNNNNNNCCNNNNNNNWWWWWWWWNNNNNN
 Structure Constraint: .......................................................
    Target Size      : 500 parts
           Lmax      : 12 bp
  Internal Repeats   : False

 Check Status: PASS

[Checking Background]
 Background: kmerSetDB stored at ./tmp_bkg/ with 21500 13-mers

 Check Status: PASS

[Checking Arguments]
   Part Type : DNA
 Struct Type : mfe
  Synth Opt  : False
   Jump Count: 10
   Fail Count: 1000
 Output File : None

 Check Status: PASS

Constructing Toolbox:

 [part] 1, [13-mers] 43, [iter time] 0.01s, [avg time] 0.01s, [total time] 0.00h
 [part] 2, [13-mers] 86, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 3, [13-mers] 129, [iter time] 0.01s, [avg time] 0.01s, [total time] 0.00h
 [part] 4, [13-mers] 172, [iter time] 0.01s, [avg time] 0.01s, [total time] 0.00h
 [part] 5, [13-mers] 215, [iter time] 0.0

 [part] 94, [13-mers] 4042, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 95, [13-mers] 4085, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 96, [13-mers] 4128, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 97, [13-mers] 4171, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 98, [13-mers] 4214, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 99, [13-mers] 4257, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 100, [13-mers] 4300, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 101, [13-mers] 4343, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 102, [13-mers] 4386, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 103, [13-mers] 4429, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 104, [13-mers] 4472, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 105, [13-mers] 4515, [iter time] 0.00s, [avg time] 0.00s, [tota

 [part] 196, [13-mers] 8428, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 197, [13-mers] 8471, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 198, [13-mers] 8514, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 199, [13-mers] 8557, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 200, [13-mers] 8600, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 201, [13-mers] 8643, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 202, [13-mers] 8686, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 203, [13-mers] 8729, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 204, [13-mers] 8772, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 205, [13-mers] 8815, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 206, [13-mers] 8858, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 207, [13-mers] 8901, [iter time] 0.01s, [avg time] 0.00s,

 [part] 297, [13-mers] 12771, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 298, [13-mers] 12814, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 299, [13-mers] 12857, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 300, [13-mers] 12900, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 301, [13-mers] 12943, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 302, [13-mers] 12986, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 303, [13-mers] 13029, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 304, [13-mers] 13072, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 305, [13-mers] 13115, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 306, [13-mers] 13158, [iter time] 0.02s, [avg time] 0.00s, [total time] 0.00h
 [part] 307, [13-mers] 13201, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 308, [13-mers] 13244, [iter time] 0.00s, [avg 

 [part] 394, [13-mers] 16942, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 395, [13-mers] 16985, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 396, [13-mers] 17028, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 397, [13-mers] 17071, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 398, [13-mers] 17114, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 399, [13-mers] 17157, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 400, [13-mers] 17200, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 401, [13-mers] 17243, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 402, [13-mers] 17286, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 403, [13-mers] 17329, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 404, [13-mers] 17372, [iter time] 0.01s, [avg time] 0.00s, [total time] 0.00h
 [part] 405, [13-mers] 17415, [iter time] 0.00s, [avg 

 [part] 491, [13-mers] 21113, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 492, [13-mers] 21156, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 493, [13-mers] 21199, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 494, [13-mers] 21242, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 495, [13-mers] 21285, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 496, [13-mers] 21328, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 497, [13-mers] 21371, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 498, [13-mers] 21414, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 499, [13-mers] 21457, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h
 [part] 500, [13-mers] 21500, [iter time] 0.00s, [avg time] 0.00s, [total time] 0.00h

Construction Complete.

Non-Repetitive Toolbox Size: 500


Let's add the new promoters to our `background` for use in subsequent sections and use `Finder` one last time to verify our design of the second toolbox.

In [48]:
bkg.multiadd(toolbox2.values()) # add all prmoters from second toolbox to background
                                # for designing parts in the subsequent sections below


[Background Processing]


  Adding Seq 268: TCCGTCACGC...

  Adding Seq 499: CAGGCACGAA...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [49]:
# recreate the list of promoters for evaluation
promoters = list(toolbox1.values()) + list(toolbox2.values())

In [50]:
# assess non-repetitiveness
non_repetitive_promoters = len(nrpcalc.finder(
    seq_list=promoters,
    Lmax=12))


[Non-Repetitive Parts Calculator - Finder Mode]

[Checking Constraints]
 Sequence List   : 1000 parts
          Lmax   : 12 bp
 Internal Repeats: False

 Check Status: PASS

[Checking Arguments]
   Vertex Cover: nrp2
   Output  File: None

 Check Status: PASS

Extracted 1000 unique sequences out of 1000 sequences in 0.0006766 seconds

Written 1000 unique sequences out to ./153a4dd7-7b81-473e-a4c5-d972aa5ba1bb/seq_list.txt in 0.001023 seconds

 [Sequence processing remaining] = 1    
 [Cliques inserted] = 1000

Built homology graph in 0.7528 seconds. [Edges = 0] [Nodes = 1000]
 [Intital Nodes = 1000] - [Repetitive Nodes = 0] = [Final Nodes = 1000]

 [+] Initial independent set = 0, computing vertex cover on remaining 0 nodes.
 [+] Vertex Cover Function: NRP 2-approximation
 [+] Dumping graph into: ./153a4dd7-7b81-473e-a4c5-d972aa5ba1bb/repeat_graph.txt in 0.007416486740112305 seconds

----------------------
Now running iteration: 0
----------------------

 Pendant checking is in progre

  [x] Isolated node 475 eliminated
  [x] Isolated node 513 eliminated
  [x] Isolated node 156 eliminated
  [x] Isolated node 93 eliminated
  [x] Isolated node 798 eliminated
  [x] Isolated node 425 eliminated
  [x] Isolated node 735 eliminated
  [x] Isolated node 362 eliminated
  [x] Isolated node 43 eliminated
  [x] Isolated node 391 eliminated
  [x] Isolated node 685 eliminated
  [x] Isolated node 312 eliminated
  [x] Isolated node 622 eliminated
  [x] Isolated node 249 eliminated
  [x] Isolated node 954 eliminated
  [x] Isolated node 891 eliminated
  [x] Isolated node 278 eliminated
  [x] Isolated node 572 eliminated
  [x] Isolated node 215 eliminated
  [x] Isolated node 904 eliminated
  [x] Isolated node 841 eliminated
  [x] Isolated node 484 eliminated
  [x] Isolated node 522 eliminated
  [x] Isolated node 165 eliminated
  [x] Isolated node 102 eliminated
  [x] Isolated node 807 eliminated
  [x] Isolated node 434 eliminated
  [x] Isolated node 728 eliminated
  [x] Isolated node 37

  [x] Isolated node 50 eliminated
  [x] Isolated node 398 eliminated
  [x] Isolated node 692 eliminated
  [x] Isolated node 335 eliminated
  [x] Isolated node 629 eliminated
  [x] Isolated node 0 eliminated
  [x] Isolated node 961 eliminated
  [x] Isolated node 642 eliminated
  [x] Isolated node 285 eliminated
  [x] Isolated node 579 eliminated
  [x] Isolated node 222 eliminated
  [x] Isolated node 927 eliminated
  [x] Isolated node 848 eliminated
  [x] Isolated node 491 eliminated
  [x] Isolated node 529 eliminated
  [x] Isolated node 172 eliminated
  [x] Isolated node 109 eliminated
  [x] Isolated node 814 eliminated
  [x] Isolated node 441 eliminated
  [x] Isolated node 751 eliminated
  [x] Isolated node 378 eliminated
  [x] Isolated node 59 eliminated
  [x] Isolated node 407 eliminated
  [x] Isolated node 701 eliminated
  [x] Isolated node 328 eliminated
  [x] Isolated node 638 eliminated
  [x] Isolated node 9 eliminated
  [x] Isolated node 970 eliminated
  [x] Isolated node 651 el

In [51]:
assert non_repetitive_promoters == 1000 # no promoters missing!

### Non-Repetititve Ribosome Binding Sites with `Lmax`=`14`

To complement our designed promoter toolboxes, we will next design a toolbox of prokaryotic ribosome binding sites (RBSs). We will primarily be using findings from [Salis et.al. (2009)](https://www.nature.com/articles/nbt.1568) for designing our RBSs.

We aim to design `1000` _de novo_ RBS sequences that are non-repetitive to our promoter toolboxes designed in the previous sections. Our RBS sequence constraint is therefore highly degenerate, containing a `28`-bp upstream region, and a `9`-bp consensus Shine-Delgarno (SD) motif (`UAAGGAGGA`) separated from the start codon (`AUG`) by a `7`-bp spacer. Importantly, the coupled structure constraint mandates a small hairpin on the `5'`-end of designed parts to insulate the RBS against fast decay, while ensuring the Shine-Delgarno motif and everything downstream remains unstructured.

Let's define and review our constraints.

In [52]:
tb3_seq_constraint = 'N'*28 + 'UAAGGAGGA' + 'N'*7 + 'AUG'
#                     -----    ---------     ----    ---
#                    SPACER    SD Motif    SPACER  START

In [53]:
tb3_struct_constraint = '.(((((....))))).............xxxxxxxxxxxxxxxxxxx'

In [54]:
print(tb3_seq_constraint)
print(tb3_struct_constraint)

NNNNNNNNNNNNNNNNNNNNNNNNNNNNUAAGGAGGANNNNNNNAUG
.(((((....))))).............xxxxxxxxxxxxxxxxxxx


The dots (`.`) in the structure constraint implies that the bases in the sequence constraint at the corresponding locations are free to either base-pair or not when a candidate part is generated. Bases marked with parenthesis (`(` and `)`) indicate that the folded structure must contain those designated base-pairings, for example the second base must pair with the fifteenth base and so on. Bases marked with `x` are forbidden from being part of any base pairing in the secondary `RNA` structure.

Before we design the RBS toolbox, we must note that the constraint for RBSs include an `Lmax` of `14` bp, whereas, the promoters were designed with an `Lmax` of `12` bp. This is because, there is a big `9`-bp constant Shine-Delgarno motif in the sequence constraint which doesn't leave too many `13`-mers (since `Lmax`=`12`) for constructing thousands of non-repetitive RBSs. As proof, let's try constructing the RBS toolbox with `Lmax`=`12`, without using the `background` developed in previous section and only using the fast `mfe` (minimum free energy) structure evaluation (a relaxed design scenerio).

In [55]:
toolbox3_attempt1 = nrpcalc.maker(
    seed=4,                                    # reproducible results
    seq_constr=tb3_seq_constraint,             # as defined above
    struct_constr=tb3_struct_constraint,       # as defined above
    Lmax=12,                                   # as stated in our goal
    target_size=1000,                          # as stated in our goal
    part_type='RNA',                           # as stated in our goal
    struct_type='mfe',                         # as defined above        
    local_model_fn=None,                       # as defined above
    global_model_fn=None,                      # as defined above
    background=None)                           # as defined above


[Non-Repetitive Parts Calculator - Maker Mode]

[Checking Constraints]
  Sequence Constraint: NNNNNNNNNNNNNNNNNNNNNNNNNNNNUAAGGAGGANNNNNNNAUG
 Structure Constraint: .(((((....))))).............xxxxxxxxxxxxxxxxxxx
    Target Size      : 1000 parts
           Lmax      : 12 bp
  Internal Repeats   : False

 [WARNING]  Target size of 1000 is unreachable from given Sequence Constraint and Lmax of 12
 [WARNING]  >> Overly constrained motifs at locations: [(24, 41)]
 [WARNING]  Fewer parts will be generated

 Check Status: PASS

[Checking Arguments]
   Part Type : RNA
 Struct Type : mfe
  Synth Opt  : False
   Jump Count: 10
   Fail Count: 1000
 Output File : None

 Check Status: PASS

Constructing Toolbox:

 [part] 1, [13-mers] 35, [iter time] 0.06s, [avg time] 0.06s, [total time] 0.00h
 [part] 2, [13-mers] 70, [iter time] 0.07s, [avg time] 0.07s, [total time] 0.00h
 [part] 3, [13-mers] 105, [iter time] 0.01s, [avg time] 0.05s, [total time] 0.00h
 [part] 4, [13-mers] 140, [iter time] 0.01s

 [part] 93, [13-mers] 3255, [iter time] 0.23s, [avg time] 0.08s, [total time] 0.00h
 [part] 94, [13-mers] 3290, [iter time] 0.01s, [avg time] 0.08s, [total time] 0.00h
 [part] 95, [13-mers] 3325, [iter time] 0.15s, [avg time] 0.08s, [total time] 0.00h
 [part] 96, [13-mers] 3360, [iter time] 0.01s, [avg time] 0.08s, [total time] 0.00h
 [part] 97, [13-mers] 3395, [iter time] 0.07s, [avg time] 0.08s, [total time] 0.00h
 [part] 98, [13-mers] 3430, [iter time] 0.12s, [avg time] 0.08s, [total time] 0.00h
 [part] 99, [13-mers] 3465, [iter time] 0.01s, [avg time] 0.08s, [total time] 0.00h
 [part] 100, [13-mers] 3500, [iter time] 0.33s, [avg time] 0.08s, [total time] 0.00h
 [part] 101, [13-mers] 3535, [iter time] 0.23s, [avg time] 0.08s, [total time] 0.00h
 [part] 102, [13-mers] 3570, [iter time] 0.04s, [avg time] 0.08s, [total time] 0.00h
 [part] 103, [13-mers] 3605, [iter time] 0.10s, [avg time] 0.08s, [total time] 0.00h
 [part] 104, [13-mers] 3640, [iter time] 0.44s, [avg time] 0.08s, [total

 [part] 190, [13-mers] 6650, [iter time] 5.04s, [avg time] 0.26s, [total time] 0.01h
 [part] 191, [13-mers] 6685, [iter time] 1.29s, [avg time] 0.27s, [total time] 0.01h
 [part] 192, [13-mers] 6720, [iter time] 0.22s, [avg time] 0.27s, [total time] 0.01h
Failure Limits Exceeded or k-mers Exhausted. Cannot Build More Parts.

Construction Complete.

Non-Repetitive Toolbox Size: 192


As we can see, `Maker` took a couple of minutes, but was only able to design `200`+ RBSs, even though we specified `target_size`=`1000`. The `9`-bp constant motif in sequence constraint leaves only `4` degenerate bases in every `13`-bp window containing the SD sequence, implying at most 4<sup>4</sup>=256 possible parts for the given sequence constraint. `Maker` was able to design `210` of those sequences before failing to find suitable _k_-mers for building further RBSs. We could try increasing the `jump_count` and/or `fail_count` to try to reach all of those `256` RBSs.

Our goal, however, is to build `1000` RBSs which is physically not possible given an `Lmax` of `12` for the specified sequence constraint. We could try introducing more degeneracy into the SD motif which might relax our constraints enough to fix the issue. But, if we don't want to alter the motif, we would have to increase our `Lmax` to expand our design space. An `Lmax` of `14` seems reasonable, giving `Maker` 4<sup>6</sup> = 4096 possible _k_-mer selection choices for all `15`-bp windows encompassing the SD motif.

Now, that we've decided to use an `Lmax`=`14` for our toolbox, how do we unify our present RBS toolbox with the previously designed promoter toolboxes, in terms of non-repetitiveness? Two approaches: (1) initialize a new `background` with `Lmax`=`14` and insert all of the promoters into it, followed by using the new `background` for designing RBSs, or (2) define a new local model function that prevents every `13`-mer in the RBSs under construction from coinciding with the previous `background` (`bkg`) containing the `13`-mers from the promoters. The first solution is pretty straight-forward, and the one we'll use for designing our toehold `RNA` switches and latter toolboxes, but for our present RBS toolbox design, we'll use the second option.

Let's look at a possible local model function for achieving the second option.

In [56]:
def prevent_promoter_conflict(seq):
    # evaluation criteria met?
    if (len(seq)) >= 13:
        # extract k-mer
        kmer = seq[-13:]
        # check for conincidence
        if kmer in bkg: # k-mer conflict found
            return (False, len(seq)-13) # retrace path
        # no conflict
        return (True, None)
    # too short a sequence
    else:
        return (True, None)

In [57]:
# toolbox1 promoter fails our evaluation as expected
assert prevent_promoter_conflict(seq=toolbox1[0]) == (False, len(toolbox1[0])-13)

In [58]:
# toolbox2 promoter also fails our evaluation as expected
assert prevent_promoter_conflict(seq=toolbox2[499]) == (False, len(toolbox1[0])-13)

In [59]:
# a poly-G 13-mer was absent in the promoters, so it is OK to be used for the RBSs
assert prevent_promoter_conflict(seq='G'*13) == (True, None)

Our local model function is done, and our `Lmax` is revised to `14`. However, unlike the previous RBS toolbox design attempt, instead of relying on just `mfe` for structure evaluation, we'll use `mfe`+`centroid`=`both` as our `struct_type` parameter to ensure both `mfe` and `centroid` conform to the given structure constraint. This ensures that the designed parts fold into the given structure with high probability (at the cost of slightly increased computation time).

In [60]:
toolbox3 = nrpcalc.maker(
    seed=5,                                    # reproducible results
    seq_constr=tb3_seq_constraint,             # as defined above
    struct_constr=tb3_struct_constraint,       # as defined above
    Lmax=14,                                   # as revised from our previous attempt
    target_size=1000,                          # as stated in our goal
    part_type='RNA',                           # as stated in our goal
    struct_type='both',                        # as revised from our previous attempt
    local_model_fn=prevent_promoter_conflict,  # as defined above
    global_model_fn=None,                      # none required
    background=None)                           # background conflict resolved via local model


[Non-Repetitive Parts Calculator - Maker Mode]

[Checking Constraints]
  Sequence Constraint: NNNNNNNNNNNNNNNNNNNNNNNNNNNNUAAGGAGGANNNNNNNAUG
 Structure Constraint: .(((((....))))).............xxxxxxxxxxxxxxxxxxx
    Target Size      : 1000 parts
           Lmax      : 14 bp
  Internal Repeats   : False

 Check Status: PASS

[Checking Arguments]
   Part Type : RNA
 Struct Type : both
  Synth Opt  : False
   Jump Count: 10
   Fail Count: 1000
 Output File : None

 Check Status: PASS

Constructing Toolbox:

 [part] 1, [15-mers] 33, [iter time] 0.03s, [avg time] 0.03s, [total time] 0.00h
 [part] 2, [15-mers] 66, [iter time] 0.36s, [avg time] 0.20s, [total time] 0.00h
 [part] 3, [15-mers] 99, [iter time] 0.10s, [avg time] 0.16s, [total time] 0.00h
 [part] 4, [15-mers] 132, [iter time] 0.01s, [avg time] 0.12s, [total time] 0.00h
 [part] 5, [15-mers] 165, [iter time] 0.02s, [avg time] 0.10s, [total time] 0.00h
 [part] 6, [15-mers] 198, [iter time] 0.11s, [avg time] 0.11s, [total time] 0.00h

 [part] 95, [15-mers] 3135, [iter time] 0.11s, [avg time] 0.10s, [total time] 0.00h
 [part] 96, [15-mers] 3168, [iter time] 0.01s, [avg time] 0.10s, [total time] 0.00h
 [part] 97, [15-mers] 3201, [iter time] 0.02s, [avg time] 0.10s, [total time] 0.00h
 [part] 98, [15-mers] 3234, [iter time] 0.02s, [avg time] 0.09s, [total time] 0.00h
 [part] 99, [15-mers] 3267, [iter time] 0.01s, [avg time] 0.09s, [total time] 0.00h
 [part] 100, [15-mers] 3300, [iter time] 0.06s, [avg time] 0.09s, [total time] 0.00h
 [part] 101, [15-mers] 3333, [iter time] 0.08s, [avg time] 0.09s, [total time] 0.00h
 [part] 102, [15-mers] 3366, [iter time] 0.01s, [avg time] 0.09s, [total time] 0.00h
 [part] 103, [15-mers] 3399, [iter time] 0.63s, [avg time] 0.10s, [total time] 0.00h
 [part] 104, [15-mers] 3432, [iter time] 0.24s, [avg time] 0.10s, [total time] 0.00h
 [part] 105, [15-mers] 3465, [iter time] 0.02s, [avg time] 0.10s, [total time] 0.00h
 [part] 106, [15-mers] 3498, [iter time] 0.09s, [avg time] 0.10s, [tot

 [part] 192, [15-mers] 6336, [iter time] 0.06s, [avg time] 0.10s, [total time] 0.01h
 [part] 193, [15-mers] 6369, [iter time] 0.02s, [avg time] 0.10s, [total time] 0.01h
 [part] 194, [15-mers] 6402, [iter time] 0.05s, [avg time] 0.10s, [total time] 0.01h
 [part] 195, [15-mers] 6435, [iter time] 0.04s, [avg time] 0.10s, [total time] 0.01h
 [part] 196, [15-mers] 6468, [iter time] 0.11s, [avg time] 0.10s, [total time] 0.01h
 [part] 197, [15-mers] 6501, [iter time] 0.45s, [avg time] 0.10s, [total time] 0.01h
 [part] 198, [15-mers] 6534, [iter time] 0.22s, [avg time] 0.10s, [total time] 0.01h
 [part] 199, [15-mers] 6567, [iter time] 0.02s, [avg time] 0.10s, [total time] 0.01h
 [part] 200, [15-mers] 6600, [iter time] 0.01s, [avg time] 0.10s, [total time] 0.01h
 [part] 201, [15-mers] 6633, [iter time] 0.03s, [avg time] 0.10s, [total time] 0.01h
 [part] 202, [15-mers] 6666, [iter time] 0.03s, [avg time] 0.10s, [total time] 0.01h
 [part] 203, [15-mers] 6699, [iter time] 0.07s, [avg time] 0.10s,

 [part] 289, [15-mers] 9537, [iter time] 0.12s, [avg time] 0.10s, [total time] 0.01h
 [part] 290, [15-mers] 9570, [iter time] 0.04s, [avg time] 0.10s, [total time] 0.01h
 [part] 291, [15-mers] 9603, [iter time] 0.21s, [avg time] 0.10s, [total time] 0.01h
 [part] 292, [15-mers] 9636, [iter time] 0.14s, [avg time] 0.10s, [total time] 0.01h
 [part] 293, [15-mers] 9669, [iter time] 0.19s, [avg time] 0.10s, [total time] 0.01h
 [part] 294, [15-mers] 9702, [iter time] 0.10s, [avg time] 0.10s, [total time] 0.01h
 [part] 295, [15-mers] 9735, [iter time] 0.03s, [avg time] 0.10s, [total time] 0.01h
 [part] 296, [15-mers] 9768, [iter time] 0.01s, [avg time] 0.10s, [total time] 0.01h
 [part] 297, [15-mers] 9801, [iter time] 0.39s, [avg time] 0.10s, [total time] 0.01h
 [part] 298, [15-mers] 9834, [iter time] 0.07s, [avg time] 0.10s, [total time] 0.01h
 [part] 299, [15-mers] 9867, [iter time] 0.44s, [avg time] 0.10s, [total time] 0.01h
 [part] 300, [15-mers] 9900, [iter time] 0.02s, [avg time] 0.10s,

 [part] 385, [15-mers] 12705, [iter time] 0.14s, [avg time] 0.10s, [total time] 0.01h
 [part] 386, [15-mers] 12738, [iter time] 0.01s, [avg time] 0.10s, [total time] 0.01h
 [part] 387, [15-mers] 12771, [iter time] 0.30s, [avg time] 0.10s, [total time] 0.01h
 [part] 388, [15-mers] 12804, [iter time] 0.18s, [avg time] 0.10s, [total time] 0.01h
 [part] 389, [15-mers] 12837, [iter time] 0.01s, [avg time] 0.10s, [total time] 0.01h
 [part] 390, [15-mers] 12870, [iter time] 0.01s, [avg time] 0.10s, [total time] 0.01h
 [part] 391, [15-mers] 12903, [iter time] 0.02s, [avg time] 0.10s, [total time] 0.01h
 [part] 392, [15-mers] 12936, [iter time] 0.05s, [avg time] 0.10s, [total time] 0.01h
 [part] 393, [15-mers] 12969, [iter time] 0.11s, [avg time] 0.10s, [total time] 0.01h
 [part] 394, [15-mers] 13002, [iter time] 0.16s, [avg time] 0.10s, [total time] 0.01h
 [part] 395, [15-mers] 13035, [iter time] 0.21s, [avg time] 0.10s, [total time] 0.01h
 [part] 396, [15-mers] 13068, [iter time] 0.49s, [avg 

...

 [part] 577, [15-mers] 19041, [iter time] 0.26s, [avg time] 0.10s, [total time] 0.02h
 [part] 578, [15-mers] 19074, [iter time] 0.24s, [avg time] 0.10s, [total time] 0.02h
 [part] 579, [15-mers] 19107, [iter time] 0.04s, [avg time] 0.10s, [total time] 0.02h
 [part] 580, [15-mers] 19140, [iter time] 0.02s, [avg time] 0.10s, [total time] 0.02h
 [part] 581, [15-mers] 19173, [iter time] 0.02s, [avg time] 0.10s, [total time] 0.02h
 [part] 582, [15-mers] 19206, [iter time] 0.04s, [avg time] 0.10s, [total time] 0.02h
 [part] 583, [15-mers] 19239, [iter time] 0.08s, [avg time] 0.10s, [total time] 0.02h
 [part] 584, [15-mers] 19272, [iter time] 0.11s, [avg time] 0.10s, [total time] 0.02h
 [part] 585, [15-mers] 19305, [iter time] 0.40s, [avg time] 0.10s, [total time] 0.02h
 [part] 586, [15-mers] 19338, [iter time] 0.03s, [avg time] 0.10s, [total time] 0.02h
 [part] 587, [15-mers] 19371, [iter time] 0.02s, [avg time] 0.10s, [total time] 0.02h
 [part] 588, [15-mers] 19404, [iter time] 0.16s, [avg 

 [part] 676, [15-mers] 22308, [iter time] 0.23s, [avg time] 0.11s, [total time] 0.02h
 [part] 677, [15-mers] 22341, [iter time] 0.09s, [avg time] 0.11s, [total time] 0.02h
 [part] 678, [15-mers] 22374, [iter time] 0.22s, [avg time] 0.11s, [total time] 0.02h
 [part] 679, [15-mers] 22407, [iter time] 0.32s, [avg time] 0.11s, [total time] 0.02h
 [part] 680, [15-mers] 22440, [iter time] 0.13s, [avg time] 0.11s, [total time] 0.02h
 [part] 681, [15-mers] 22473, [iter time] 0.02s, [avg time] 0.11s, [total time] 0.02h
 [part] 682, [15-mers] 22506, [iter time] 0.08s, [avg time] 0.11s, [total time] 0.02h
 [part] 683, [15-mers] 22539, [iter time] 0.01s, [avg time] 0.11s, [total time] 0.02h
 [part] 684, [15-mers] 22572, [iter time] 0.30s, [avg time] 0.11s, [total time] 0.02h
 [part] 685, [15-mers] 22605, [iter time] 0.30s, [avg time] 0.11s, [total time] 0.02h
 [part] 686, [15-mers] 22638, [iter time] 0.18s, [avg time] 0.11s, [total time] 0.02h
 [part] 687, [15-mers] 22671, [iter time] 0.03s, [avg 

 [part] 775, [15-mers] 25575, [iter time] 0.28s, [avg time] 0.12s, [total time] 0.03h
 [part] 776, [15-mers] 25608, [iter time] 0.19s, [avg time] 0.12s, [total time] 0.03h
 [part] 777, [15-mers] 25641, [iter time] 0.20s, [avg time] 0.12s, [total time] 0.03h
 [part] 778, [15-mers] 25674, [iter time] 0.20s, [avg time] 0.12s, [total time] 0.03h
 [part] 779, [15-mers] 25707, [iter time] 0.23s, [avg time] 0.12s, [total time] 0.03h
 [part] 780, [15-mers] 25740, [iter time] 0.40s, [avg time] 0.12s, [total time] 0.03h
 [part] 781, [15-mers] 25773, [iter time] 0.02s, [avg time] 0.12s, [total time] 0.03h
 [part] 782, [15-mers] 25806, [iter time] 0.10s, [avg time] 0.12s, [total time] 0.03h
 [part] 783, [15-mers] 25839, [iter time] 0.05s, [avg time] 0.12s, [total time] 0.03h
 [part] 784, [15-mers] 25872, [iter time] 0.03s, [avg time] 0.12s, [total time] 0.03h
 [part] 785, [15-mers] 25905, [iter time] 0.13s, [avg time] 0.12s, [total time] 0.03h
 [part] 786, [15-mers] 25938, [iter time] 0.21s, [avg 

 [part] 872, [15-mers] 28776, [iter time] 0.13s, [avg time] 0.12s, [total time] 0.03h
 [part] 873, [15-mers] 28809, [iter time] 0.04s, [avg time] 0.12s, [total time] 0.03h
 [part] 874, [15-mers] 28842, [iter time] 0.08s, [avg time] 0.12s, [total time] 0.03h
 [part] 875, [15-mers] 28875, [iter time] 0.10s, [avg time] 0.12s, [total time] 0.03h
 [part] 876, [15-mers] 28908, [iter time] 0.04s, [avg time] 0.12s, [total time] 0.03h
 [part] 877, [15-mers] 28941, [iter time] 0.39s, [avg time] 0.12s, [total time] 0.03h
 [part] 878, [15-mers] 28974, [iter time] 0.16s, [avg time] 0.12s, [total time] 0.03h
 [part] 879, [15-mers] 29007, [iter time] 0.04s, [avg time] 0.12s, [total time] 0.03h
 [part] 880, [15-mers] 29040, [iter time] 0.08s, [avg time] 0.12s, [total time] 0.03h
 [part] 881, [15-mers] 29073, [iter time] 0.01s, [avg time] 0.12s, [total time] 0.03h
 [part] 882, [15-mers] 29106, [iter time] 0.30s, [avg time] 0.12s, [total time] 0.03h
 [part] 883, [15-mers] 29139, [iter time] 0.06s, [avg 

 [part] 968, [15-mers] 31944, [iter time] 0.17s, [avg time] 0.13s, [total time] 0.04h
 [part] 969, [15-mers] 31977, [iter time] 0.36s, [avg time] 0.13s, [total time] 0.04h
 [part] 970, [15-mers] 32010, [iter time] 0.57s, [avg time] 0.13s, [total time] 0.04h
 [part] 971, [15-mers] 32043, [iter time] 0.01s, [avg time] 0.13s, [total time] 0.04h
 [part] 972, [15-mers] 32076, [iter time] 0.71s, [avg time] 0.13s, [total time] 0.04h
 [part] 973, [15-mers] 32109, [iter time] 0.18s, [avg time] 0.13s, [total time] 0.04h
 [part] 974, [15-mers] 32142, [iter time] 0.08s, [avg time] 0.13s, [total time] 0.04h
 [part] 975, [15-mers] 32175, [iter time] 0.38s, [avg time] 0.13s, [total time] 0.04h
 [part] 976, [15-mers] 32208, [iter time] 0.05s, [avg time] 0.13s, [total time] 0.04h
 [part] 977, [15-mers] 32241, [iter time] 0.17s, [avg time] 0.13s, [total time] 0.04h
 [part] 978, [15-mers] 32274, [iter time] 0.04s, [avg time] 0.13s, [total time] 0.04h
 [part] 979, [15-mers] 32307, [iter time] 0.31s, [avg 

In [61]:
toolbox3[0] # first RBS designed

'CCGATGTGCACATCGTAGTGGAATAAGTTAAGGAGGACGTTTTAATG'

In [62]:
toolbox3[999] # last RBS designed

'AGCGAAGGGCTTCGCTCATAGAAAGCTATAAGGAGGAATTTGGAATG'

In [63]:
assert len(toolbox3) == 1000 # our goal of 1000 brand new non-repetitive RBSs is met!

### Non-Repetitive Toehold Switches with `Lmax`=`14`

We will now design $1000$ non-repetitive toehold RNA switches for programmable protein expression. Our constraints for designing these toehold `RNA` switches are based on the work of [Green et. al. (2014)](https://www.sciencedirect.com/science/article/pii/S0092867414012896).

Before we embark on the design, let's delete the previous `background` (`bkg`) and initialize a new `background` with `Lmax`=`14`, into which we'll insert all of the previous toolboxes, so that we can use it for designing our toehold `RNA` switches non-repetitive to all previously designed parts.

In [64]:
bkg.drop() # deletes the background from disk

True

In [65]:
# chained_bkg.drop()

In [66]:
chained_bkg = nrpcalc.background(
    path='./chained_bkg', # a new background path
    Lmax=14)              # updated Lmax

In [67]:
chained_bkg # check new background path and content

kmerSetDB stored at ./chained_bkg/ with 0 15-mers

In [68]:
chained_bkg.multiadd(toolbox1.values()) # add the first promoter toolbox
chained_bkg.multiadd(toolbox1.values()) # add the second promoter toolbox
chained_bkg.multiadd(toolbox1.values()) # add the third toolbox containing RBSs


[Background Processing]


  Adding Seq 268: GCCCGATTGA...

  Adding Seq 499: CGGGGGTCCG...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 268: GCCCGATTGA...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 499: CGGGGGTCCG...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 268: GCCCGATTGA...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 499: CGGGGGTCCG...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [69]:
chained_bkg # review background post insertion

kmerSetDB stored at ./chained_bkg/ with 20500 15-mers

Our constraint for toehold `RNA` switches primarily includes a hairpin loop, and containing a `30`-bp **trigger RNA** sequence usptream of an embedded `7`-bp consensus Shine-Delgarno motif (`AGGAGGA`), separated from the start codon (`AUG`) by the remaining `6`-bp stem of domain _B_, and ends with a `21`-bp linker sequence. Notably, everything upstream of the linker portion of the design has a very specific structure requirement.

Let's define the sequence and structure constraints and review them.

In [70]:
tb4_seq_constraint = 'N'*12 + 'N'*9 + 'N'*3 + 'N'*6 + 'AGGAGGA' + 'N'*6 + 'AUG' + 'N'*9 + 'N'*21
#                     -----    --------------------    -------     --------------------    -----
#                  Domain A     Domain B with Bulge    SD Motif    Domain B with START     LINKER

In [71]:
tb4_struct_constraint = 'xxxxxxxxxxxx(((((((((xxx((((((xxxxxxx))))))xxx))))))))).....................'

In [72]:
print('{:^30}'.format('Trigger RNA Sequence'))
print('-'*30)
print(tb4_seq_constraint)
print(tb4_struct_constraint)

     Trigger RNA Sequence     
------------------------------
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNAGGAGGANNNNNNAUGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
xxxxxxxxxxxx(((((((((xxx((((((xxxxxxx))))))xxx))))))))).....................


Although, at first glance, the constraints look fine and degenerate enough, there is a potential pitfall waiting for us when we feed these constraints to `Maker`.

Notice, that the `7`-bp SD motif (`AGGAGGA`) is flanked by domain _B_ bases on either side, which engenders a `15`-bp _k_-mer window with `AGGAGGA` in the middle and four paired bases on either side. This is illustrated below.

In this `15`-mer window, the last four bases are always going to be complementary to the first four bases. So, as soon as the first four bases (`N`s) are filled in by `Maker`, the fate of the last four bases are automatically determined (they will be complementary to the first four bases). The `7`-bp SD motif is a constant in our sequence constraint which leaves only the first four bases to be selected variably by `Maker` (the last four bases become a dynamically inserted constant for each imaginable run). Thus, instead of working with a degenerate `15`-bp window with `7` bases fixed, we're actually working with a `15`-bp window with `7`+`4`=`11` bases fixed. This leaves us with $4^4 = 256$ possible nucleotide combinations to fill up this window, implying, a theoretical maximum toolbox size of only `256` toehold `RNA` switches. Our goal for `1000` non-repetitive toehold RNA switches, even with `Lmax`=`14`, will not be fulfilled given how we have framed the sequence and structure constraint.

Rather than abandoning hope, we may go back to the original paper for insights. It is clear that an RBS must be located between the two halves of domain _B_, but should this RBS just consist of a consensus `7`-bp SD motif only? The motif can potentially be "padded" on either side, and still leave us with effective RBSs. Accordingly, we will modify our sequence constraint to pad the SD motif with three `N`s on the 5' end, while ensuring that those bases remain unpaired via our modified structure constraint. This expands our design space by sixty four fold, making our goal of `1000` non-repetitive toehold `RNA` switches a possibility. Let's re-define the constraints, and review them one last time.

In [73]:
tb4_seq_constraint = 'N'*12 + 'N'*9 + 'N'*3 + 'N'*6 + 'NNNAGGAGGA' + 'N'*6 + 'AUG' + 'N'*9 + 'N'*21
#                     -----    --------------------    ----------     --------------------    -----
#                  Domain A     Domain B with Bulge    Padded SD      Domain B with START     LINKER

In [74]:
tb4_struct_constraint = 'xxxxxxxxxxxx(((((((((xxx((((((xxxxxxxxxx))))))xxx))))))))).....................'

In [75]:
print('-'*30 + ' <-- Trigger RNA Sequence')
print(tb4_seq_constraint)
print(tb4_struct_constraint)

------------------------------ <-- Trigger RNA Sequence
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNAGGAGGANNNNNNAUGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
xxxxxxxxxxxx(((((((((xxx((((((xxxxxxxxxx))))))xxx))))))))).....................


Because we're dealing with toehold `RNA` switches, and the design includes `N`s after the start codon, it is imperative to prevent any in-frame stop codons immediately after the start codon in the linker region. Similarly, it is important to prevent any start codon after the SD motif, in the 9 bp spacer before the designated start codon. Time for a quick local model function.

> **Note** `Maker` builds and returns `DNA` strings regardless of the input `part_type` specficiation, because the generated parts are usually synthesiszed as `DNA` for cloning. The `part_type` is used to ensure correct base pairing, and select the correct energy parameters for evaluating the structure constraint in the intended scenerio. For example, toehold RNA switches are designed using correct parameters so that when they finally fold in their RNA state, they have the correct conformation. This also means that all local and global model functions should therefore evaluate `DNA` strings for evaluation at all times.

In [76]:
start_codon = 'ATG' # rather than 'AUG'

In [77]:
stop_codons = set(['TAG', 'TAA', 'TGA']) # all stop codons are defined

In [113]:
def prevent_codon(seq):
    # we don't evaluate if were're at or before SD motif, or at
    # the designated start codon location and the the two bases
    # right after the start codon (which do not form an in-frame codon)
    
    # case 1: at or before SD motif
    if len(seq) <= 40: # pass evaluation
        return (True, None)
    
    # case 2: at the designated start codon or the two
    # bases right next to it
    if 47 <= len(seq) <= 49+2: # pass evaluation
        return (True, None)
    
    # actual evaluation time!
    
    # case 1: we have entered in the spacer after start codon
    if 41 <= len(seq) <= 46:
        # extract codon candidate
        cdn = seq[-3:]
        # is this a start codon?
        if cdn == start_codon:
            return (False, len(seq)-3) # go back three places
        # not a start codon
        return (True, None)
    
    # case 2: we have entered the linker region beyond the start codon
    if len(seq) >= 52: # first in-frame codon after the start codon
        # extract codon candidate
        cdn = seq[-3:]
        # is the codon candidate a stop codon?
        if cdn in stop_codons:
            return (False, len(seq)-3) # go back three places
        # candidate is not a stop codon
        return (True, None) # pass

In [114]:
assert prevent_codon(seq='A'*25) == (True, None) # short sequences pass

In [115]:
assert prevent_codon(seq='A'*40 + 'ATG') == (False, 40) # start codon in spacer after SD prevented

In [116]:
assert prevent_codon(seq='A'*46 + 'ATG') == (True, None) # start codon at designated location not evaluated

In [117]:
assert prevent_codon(seq='A'*50 + 'TAA') == (False, 50) # stop codons after start codon prevented

In [118]:
assert prevent_codon(seq='A'*50 + 'GCA') == (True, None) # other codons are fine

Let's now build our brand new toolbox of non-repetitive toehold `RNA` switches and review them!

In [119]:
toolbox4 = nrpcalc.maker(
    seed=6,                                    # reproducible results
    seq_constr=tb4_seq_constraint,             # as defined above
    struct_constr=tb4_struct_constraint,       # as defined above
    Lmax=14,                                   # as defined above
    target_size=1000,                          # as stated in our goal
    part_type='RNA',                           # as stated in our goal
    struct_type='both',                        # as stated in our goal
    local_model_fn=prevent_codon,              # as defined above
    global_model_fn=None,                      # no requirement of a global check
    background=chained_bkg)                    # as defined above


[Non-Repetitive Parts Calculator - Maker Mode]

[Checking Constraints]
  Sequence Constraint: NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNAGGAGGANNNNNNAUGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
 Structure Constraint: xxxxxxxxxxxx(((((((((xxx((((((xxxxxxxxxx))))))xxx))))))))).....................
    Target Size      : 1000 parts
           Lmax      : 14 bp
  Internal Repeats   : False

 Check Status: PASS

[Checking Background]
 Background: kmerSetDB stored at ./chained_bkg/ with 20500 15-mers

 Check Status: PASS

[Checking Arguments]
   Part Type : RNA
 Struct Type : both
  Synth Opt  : False
   Jump Count: 10
   Fail Count: 1000
 Output File : None

 Check Status: PASS

Constructing Toolbox:

 [part] 1, [15-mers] 65, [iter time] 0.21s, [avg time] 0.21s, [total time] 0.00h
 [part] 2, [15-mers] 130, [iter time] 0.22s, [avg time] 0.22s, [total time] 0.00h
 [part] 3, [15-mers] 195, [iter time] 0.55s, [avg time] 0.33s, [total time] 0.00h
 [part] 4, [15-mers] 260, [iter time] 0.05s, [avg time] 0.26s, [tota

 [part] 91, [15-mers] 5915, [iter time] 0.16s, [avg time] 0.21s, [total time] 0.01h
 [part] 92, [15-mers] 5980, [iter time] 0.05s, [avg time] 0.21s, [total time] 0.01h
 [part] 93, [15-mers] 6045, [iter time] 0.47s, [avg time] 0.22s, [total time] 0.01h
 [part] 94, [15-mers] 6110, [iter time] 0.43s, [avg time] 0.22s, [total time] 0.01h
 [part] 95, [15-mers] 6175, [iter time] 0.11s, [avg time] 0.22s, [total time] 0.01h
 [part] 96, [15-mers] 6240, [iter time] 0.03s, [avg time] 0.21s, [total time] 0.01h
 [part] 97, [15-mers] 6305, [iter time] 0.03s, [avg time] 0.21s, [total time] 0.01h
 [part] 98, [15-mers] 6370, [iter time] 0.06s, [avg time] 0.21s, [total time] 0.01h
 [part] 99, [15-mers] 6435, [iter time] 0.79s, [avg time] 0.22s, [total time] 0.01h
 [part] 100, [15-mers] 6500, [iter time] 0.04s, [avg time] 0.22s, [total time] 0.01h
 [part] 101, [15-mers] 6565, [iter time] 0.04s, [avg time] 0.21s, [total time] 0.01h
 [part] 102, [15-mers] 6630, [iter time] 0.02s, [avg time] 0.21s, [total t

 [part] 188, [15-mers] 12220, [iter time] 1.53s, [avg time] 0.22s, [total time] 0.01h
 [part] 189, [15-mers] 12285, [iter time] 0.17s, [avg time] 0.22s, [total time] 0.01h
 [part] 190, [15-mers] 12350, [iter time] 0.06s, [avg time] 0.22s, [total time] 0.01h
 [part] 191, [15-mers] 12415, [iter time] 0.15s, [avg time] 0.22s, [total time] 0.01h
 [part] 192, [15-mers] 12480, [iter time] 0.21s, [avg time] 0.22s, [total time] 0.01h
 [part] 193, [15-mers] 12545, [iter time] 0.76s, [avg time] 0.22s, [total time] 0.01h
 [part] 194, [15-mers] 12610, [iter time] 0.18s, [avg time] 0.22s, [total time] 0.01h
 [part] 195, [15-mers] 12675, [iter time] 0.16s, [avg time] 0.22s, [total time] 0.01h
 [part] 196, [15-mers] 12740, [iter time] 0.05s, [avg time] 0.22s, [total time] 0.01h
 [part] 197, [15-mers] 12805, [iter time] 0.79s, [avg time] 0.22s, [total time] 0.01h
 [part] 198, [15-mers] 12870, [iter time] 0.17s, [avg time] 0.22s, [total time] 0.01h
 [part] 199, [15-mers] 12935, [iter time] 0.10s, [avg 

 [part] 284, [15-mers] 18460, [iter time] 0.38s, [avg time] 0.21s, [total time] 0.02h
 [part] 285, [15-mers] 18525, [iter time] 0.38s, [avg time] 0.21s, [total time] 0.02h
 [part] 286, [15-mers] 18590, [iter time] 0.05s, [avg time] 0.21s, [total time] 0.02h
 [part] 287, [15-mers] 18655, [iter time] 0.24s, [avg time] 0.21s, [total time] 0.02h
 [part] 288, [15-mers] 18720, [iter time] 0.30s, [avg time] 0.21s, [total time] 0.02h
 [part] 289, [15-mers] 18785, [iter time] 0.29s, [avg time] 0.21s, [total time] 0.02h
 [part] 290, [15-mers] 18850, [iter time] 0.04s, [avg time] 0.21s, [total time] 0.02h
 [part] 291, [15-mers] 18915, [iter time] 0.20s, [avg time] 0.21s, [total time] 0.02h
 [part] 292, [15-mers] 18980, [iter time] 0.08s, [avg time] 0.21s, [total time] 0.02h
 [part] 293, [15-mers] 19045, [iter time] 0.28s, [avg time] 0.21s, [total time] 0.02h
 [part] 294, [15-mers] 19110, [iter time] 0.06s, [avg time] 0.21s, [total time] 0.02h
 [part] 295, [15-mers] 19175, [iter time] 0.10s, [avg 

 [part] 380, [15-mers] 24700, [iter time] 0.45s, [avg time] 0.22s, [total time] 0.02h
 [part] 381, [15-mers] 24765, [iter time] 0.07s, [avg time] 0.22s, [total time] 0.02h
 [part] 382, [15-mers] 24830, [iter time] 0.18s, [avg time] 0.22s, [total time] 0.02h
 [part] 383, [15-mers] 24895, [iter time] 0.17s, [avg time] 0.22s, [total time] 0.02h
 [part] 384, [15-mers] 24960, [iter time] 0.03s, [avg time] 0.22s, [total time] 0.02h
 [part] 385, [15-mers] 25025, [iter time] 0.03s, [avg time] 0.22s, [total time] 0.02h
 [part] 386, [15-mers] 25090, [iter time] 0.04s, [avg time] 0.22s, [total time] 0.02h
 [part] 387, [15-mers] 25155, [iter time] 0.24s, [avg time] 0.22s, [total time] 0.02h
 [part] 388, [15-mers] 25220, [iter time] 0.03s, [avg time] 0.22s, [total time] 0.02h
 [part] 389, [15-mers] 25285, [iter time] 0.15s, [avg time] 0.22s, [total time] 0.02h
 [part] 390, [15-mers] 25350, [iter time] 0.03s, [avg time] 0.22s, [total time] 0.02h
 [part] 391, [15-mers] 25415, [iter time] 0.26s, [avg 

...

 [part] 575, [15-mers] 37375, [iter time] 0.25s, [avg time] 0.22s, [total time] 0.03h
 [part] 576, [15-mers] 37440, [iter time] 1.02s, [avg time] 0.22s, [total time] 0.03h
 [part] 577, [15-mers] 37505, [iter time] 0.16s, [avg time] 0.22s, [total time] 0.03h
 [part] 578, [15-mers] 37570, [iter time] 0.34s, [avg time] 0.22s, [total time] 0.03h
 [part] 579, [15-mers] 37635, [iter time] 0.15s, [avg time] 0.22s, [total time] 0.03h
 [part] 580, [15-mers] 37700, [iter time] 0.03s, [avg time] 0.22s, [total time] 0.03h
 [part] 581, [15-mers] 37765, [iter time] 0.11s, [avg time] 0.22s, [total time] 0.03h
 [part] 582, [15-mers] 37830, [iter time] 0.03s, [avg time] 0.22s, [total time] 0.03h
 [part] 583, [15-mers] 37895, [iter time] 0.05s, [avg time] 0.22s, [total time] 0.03h
 [part] 584, [15-mers] 37960, [iter time] 0.07s, [avg time] 0.22s, [total time] 0.03h
 [part] 585, [15-mers] 38025, [iter time] 0.36s, [avg time] 0.22s, [total time] 0.04h
 [part] 586, [15-mers] 38090, [iter time] 0.34s, [avg 

 [part] 671, [15-mers] 43615, [iter time] 0.44s, [avg time] 0.22s, [total time] 0.04h
 [part] 672, [15-mers] 43680, [iter time] 0.33s, [avg time] 0.22s, [total time] 0.04h
 [part] 673, [15-mers] 43745, [iter time] 0.21s, [avg time] 0.22s, [total time] 0.04h
 [part] 674, [15-mers] 43810, [iter time] 0.16s, [avg time] 0.22s, [total time] 0.04h
 [part] 675, [15-mers] 43875, [iter time] 0.49s, [avg time] 0.22s, [total time] 0.04h
 [part] 676, [15-mers] 43940, [iter time] 0.13s, [avg time] 0.22s, [total time] 0.04h
 [part] 677, [15-mers] 44005, [iter time] 0.07s, [avg time] 0.22s, [total time] 0.04h
 [part] 678, [15-mers] 44070, [iter time] 0.16s, [avg time] 0.22s, [total time] 0.04h
 [part] 679, [15-mers] 44135, [iter time] 0.77s, [avg time] 0.22s, [total time] 0.04h
 [part] 680, [15-mers] 44200, [iter time] 0.16s, [avg time] 0.22s, [total time] 0.04h
 [part] 681, [15-mers] 44265, [iter time] 0.11s, [avg time] 0.22s, [total time] 0.04h
 [part] 682, [15-mers] 44330, [iter time] 0.12s, [avg 

 [part] 767, [15-mers] 49855, [iter time] 0.27s, [avg time] 0.22s, [total time] 0.05h
 [part] 768, [15-mers] 49920, [iter time] 0.49s, [avg time] 0.22s, [total time] 0.05h
 [part] 769, [15-mers] 49985, [iter time] 0.07s, [avg time] 0.22s, [total time] 0.05h
 [part] 770, [15-mers] 50050, [iter time] 0.05s, [avg time] 0.22s, [total time] 0.05h
 [part] 771, [15-mers] 50115, [iter time] 0.10s, [avg time] 0.22s, [total time] 0.05h
 [part] 772, [15-mers] 50180, [iter time] 0.09s, [avg time] 0.22s, [total time] 0.05h
 [part] 773, [15-mers] 50245, [iter time] 0.59s, [avg time] 0.22s, [total time] 0.05h
 [part] 774, [15-mers] 50310, [iter time] 0.05s, [avg time] 0.22s, [total time] 0.05h
 [part] 775, [15-mers] 50375, [iter time] 0.14s, [avg time] 0.22s, [total time] 0.05h
 [part] 776, [15-mers] 50440, [iter time] 1.56s, [avg time] 0.22s, [total time] 0.05h
 [part] 777, [15-mers] 50505, [iter time] 0.66s, [avg time] 0.22s, [total time] 0.05h
 [part] 778, [15-mers] 50570, [iter time] 0.70s, [avg 

 [part] 864, [15-mers] 56160, [iter time] 0.24s, [avg time] 0.23s, [total time] 0.05h
 [part] 865, [15-mers] 56225, [iter time] 0.38s, [avg time] 0.23s, [total time] 0.05h
 [part] 866, [15-mers] 56290, [iter time] 0.13s, [avg time] 0.23s, [total time] 0.05h
 [part] 867, [15-mers] 56355, [iter time] 1.70s, [avg time] 0.23s, [total time] 0.05h
 [part] 868, [15-mers] 56420, [iter time] 0.25s, [avg time] 0.23s, [total time] 0.05h
 [part] 869, [15-mers] 56485, [iter time] 0.04s, [avg time] 0.23s, [total time] 0.05h
 [part] 870, [15-mers] 56550, [iter time] 0.28s, [avg time] 0.23s, [total time] 0.06h
 [part] 871, [15-mers] 56615, [iter time] 0.10s, [avg time] 0.23s, [total time] 0.06h
 [part] 872, [15-mers] 56680, [iter time] 0.20s, [avg time] 0.23s, [total time] 0.06h
 [part] 873, [15-mers] 56745, [iter time] 0.33s, [avg time] 0.23s, [total time] 0.06h
 [part] 874, [15-mers] 56810, [iter time] 0.78s, [avg time] 0.23s, [total time] 0.06h
 [part] 875, [15-mers] 56875, [iter time] 0.04s, [avg 

 [part] 960, [15-mers] 62400, [iter time] 0.57s, [avg time] 0.23s, [total time] 0.06h
 [part] 961, [15-mers] 62465, [iter time] 0.10s, [avg time] 0.23s, [total time] 0.06h
 [part] 962, [15-mers] 62530, [iter time] 0.58s, [avg time] 0.23s, [total time] 0.06h
 [part] 963, [15-mers] 62595, [iter time] 0.06s, [avg time] 0.23s, [total time] 0.06h
 [part] 964, [15-mers] 62660, [iter time] 0.06s, [avg time] 0.23s, [total time] 0.06h
 [part] 965, [15-mers] 62725, [iter time] 0.33s, [avg time] 0.23s, [total time] 0.06h
 [part] 966, [15-mers] 62790, [iter time] 0.04s, [avg time] 0.23s, [total time] 0.06h
 [part] 967, [15-mers] 62855, [iter time] 0.32s, [avg time] 0.23s, [total time] 0.06h
 [part] 968, [15-mers] 62920, [iter time] 0.20s, [avg time] 0.23s, [total time] 0.06h
 [part] 969, [15-mers] 62985, [iter time] 0.64s, [avg time] 0.23s, [total time] 0.06h
 [part] 970, [15-mers] 63050, [iter time] 0.26s, [avg time] 0.23s, [total time] 0.06h
 [part] 971, [15-mers] 63115, [iter time] 0.09s, [avg 

In [121]:
toolbox4[0] # first switch in the toolbox

'ACTATGTCTCTCGCGATTCCGGTCCGCGGTCGTAGGAGGAACCGCGATGCGGAATCGCTTTCTCTCGCGCAACCAATGG'

In [126]:
toolbox4[999] # last switch in the toolbox

'AATCAAATACACCGCAATATAAGGCGTCCGGAGAGGAGGACGGACGATGTATATTGCGTCGGAGACATGGTCAGAGCTT'

In [125]:
assert len(nrpcalc.finder(
    seq_list=toolbox4.values(),
    Lmax=14,
    background=chained_bkg)) == 1000 # job done!


[Non-Repetitive Parts Calculator - Finder Mode]

[Checking Constraints]
 Sequence List   : 1000 parts
          Lmax   : 14 bp
 Internal Repeats: False

 Check Status: PASS

[Checking Background]
 Background: kmerSetDB stored at ./chained_bkg/ with 20500 15-mers

 Check Status: PASS

[Checking Arguments]
   Vertex Cover: nrp2
   Output  File: None

 Check Status: PASS

Extracted 1000 unique sequences out of 1000 sequences in 0.003117 seconds

Written 1000 unique sequences out to ./7edf7751-0bdd-4735-88de-5618a2156817/seq_list.txt in 0.003076 seconds

 [Sequence processing remaining] = 1    
 [Cliques inserted] = 1000

Built homology graph in 1.211 seconds. [Edges = 0] [Nodes = 1000]
 [Intital Nodes = 1000] - [Repetitive Nodes = 0] = [Final Nodes = 1000]

 [+] Initial independent set = 0, computing vertex cover on remaining 0 nodes.
 [+] Vertex Cover Function: NRP 2-approximation
 [+] Dumping graph into: ./7edf7751-0bdd-4735-88de-5618a2156817/repeat_graph.txt in 0.0037260055541992188 s

  [x] Isolated node 208 eliminated
  [x] Isolated node 913 eliminated
  [x] Isolated node 850 eliminated
  [x] Isolated node 493 eliminated
  [x] Isolated node 531 eliminated
  [x] Isolated node 174 eliminated
  [x] Isolated node 111 eliminated
  [x] Isolated node 800 eliminated
  [x] Isolated node 443 eliminated
  [x] Isolated node 737 eliminated
  [x] Isolated node 380 eliminated
  [x] Isolated node 61 eliminated
  [x] Isolated node 393 eliminated
  [x] Isolated node 703 eliminated
  [x] Isolated node 330 eliminated
  [x] Isolated node 624 eliminated
  [x] Isolated node 11 eliminated
  [x] Isolated node 972 eliminated
  [x] Isolated node 653 eliminated
  [x] Isolated node 280 eliminated
  [x] Isolated node 590 eliminated
  [x] Isolated node 217 eliminated
  [x] Isolated node 922 eliminated
  [x] Isolated node 859 eliminated
  [x] Isolated node 502 eliminated
  [x] Isolated node 540 eliminated
  [x] Isolated node 183 eliminated
  [x] Isolated node 104 eliminated
  [x] Isolated node 80

  [x] Isolated node 272 eliminated
  [x] Isolated node 582 eliminated
  [x] Isolated node 209 eliminated
  [x] Isolated node 914 eliminated
  [x] Isolated node 851 eliminated
  [x] Isolated node 494 eliminated
  [x] Isolated node 532 eliminated
  [x] Isolated node 175 eliminated
  [x] Isolated node 96 eliminated
  [x] Isolated node 801 eliminated
  [x] Isolated node 444 eliminated
  [x] Isolated node 738 eliminated
  [x] Isolated node 381 eliminated
  [x] Isolated node 62 eliminated
  [x] Isolated node 394 eliminated
  [x] Isolated node 688 eliminated
  [x] Isolated node 331 eliminated
  [x] Isolated node 625 eliminated
  [x] Isolated node 12 eliminated
  [x] Isolated node 973 eliminated
  [x] Isolated node 654 eliminated
  [x] Isolated node 281 eliminated
  [x] Isolated node 591 eliminated
  [x] Isolated node 218 eliminated
  [x] Isolated node 923 eliminated
  [x] Isolated node 860 eliminated
  [x] Isolated node 503 eliminated
  [x] Isolated node 541 eliminated
  [x] Isolated node 168

  [x] Isolated node 524 eliminated
  [x] Isolated node 167 eliminated
  [x] Isolated node 88 eliminated
  [x] Isolated node 793 eliminated
  [x] Isolated node 436 eliminated
  [x] Isolated node 730 eliminated
  [x] Isolated node 373 eliminated
  [x] Isolated node 54 eliminated
  [x] Isolated node 386 eliminated
  [x] Isolated node 680 eliminated
  [x] Isolated node 323 eliminated
  [x] Isolated node 617 eliminated
  [x] Isolated node 4 eliminated
  [x] Isolated node 965 eliminated
  [x] Isolated node 646 eliminated
  [x] Isolated node 273 eliminated
  [x] Isolated node 583 eliminated
  [x] Isolated node 210 eliminated
  [x] Isolated node 915 eliminated
  [x] Isolated node 852 eliminated
  [x] Isolated node 495 eliminated
  [x] Isolated node 533 eliminated
  [x] Isolated node 160 eliminated
  [x] Isolated node 97 eliminated
  [x] Isolated node 802 eliminated
  [x] Isolated node 445 eliminated
  [x] Isolated node 739 eliminated
  [x] Isolated node 382 eliminated
  [x] Isolated node 63 el

### Non-Repetitive Intrinsic Terminators with `Lmax`=`14`

In [ ]:
tb5seq = 'N'*1
tb5str = '.......(((((((((((((((((((((((((........)))))))))))))))))))))))))....................'

In [ ]:
len(tb5str)